In [1]:
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import os, sys 

os.chdir('/home/aivar/projects/ssa-solvers') # project directory

%load_ext line_profiler
%matplotlib inline

torch.set_default_tensor_type(torch.FloatTensor)

device = torch.device('cuda:0') #if torch.cuda.is_available() else 
# device = torch.device("cpu")


### testing direct vs first reaction

In [2]:
from circuits.mrna_srna.mrna_srna_intrans import mRNAsRNAInTrans, cfg

from ssa_solvers.simulators import StochasticSimulator, DeterministicSimulator 
from ssa_solvers.data_class import SimulationData

end_time = 300
n_steps = 300
n_traj = 20000


In [3]:
reaction_system = mRNAsRNAInTrans(device=device)
init_pops = torch.zeros((reaction_system.n_species, ), dtype=torch.int64, device=device) #torch.randint(1, (reaction_system.n_species, ))
time_grid = np.arange(0, end_time, int(end_time / n_steps))

# Specifying a simulator using direct method
cfg['solver'] = 'direct'
ssa_simulator_direct = StochasticSimulator(
    reaction_system=reaction_system,
    cfg=cfg,
    device=device
)
# Specifying a simulator using first reaction method
cfg['solver'] = 'first_reaction'
ssa_simulator_fr = StochasticSimulator(
    reaction_system=reaction_system,
    cfg=cfg,
    device=device
)

In [ ]:
%timeit ssa_simulator_fr.simulate(init_pops=init_pops, end_time=end_time, n_trajectories=n_traj)
%timeit ssa_simulator_direct.simulate(init_pops=init_pops, end_time=end_time, n_trajectories=n_traj)

Parameters for deterministic simulations 

In [ ]:
reaction_system = mRNAsRNAInTrans()
ode_simulator = DeterministicSimulator(
    reaction_system=reaction_system,
    cfg=cfg
)
init_pops = np.zeros((reaction_system.n_species,))
time_grid_np = np.arange(0, end_time, int(end_time / n_steps))
ode_res = ode_simulator.simulate(init_pops=init_pops, time_grid=time_grid)

### Plotting results for verification

Comptuting the statistics on a grid

In [ ]:
means_direct, stds_direct = ssa_simulator_direct.data_set_direct.mean_and_set(time_grid=time_grid)
means_fr, stds_fr = ssa_simulator_fr.data_set_fr.mean_and_set(time_grid=time_grid)

Plotting

In [ ]:
species_idx = 1
plt.figure()
# ode
plt.plot(time_grid, ode_res[species_idx, :], 'k', label='ODE')
# direct ssa 
plt.plot(time_grid, means_direct[species_idx, :], 'b', label='Mean SSA (direct)')
plt.fill_between(time_grid, means_direct[species_idx,:]+stds_direct[species_idx, :], means_direct[species_idx,:]-stds_direct[species_idx, :], 
                 color='b', alpha=0.3)
plt.xlim([0, end_time])    

plt.xlabel('Time (s)')
plt.ylabel('Number of species')
# first reaction ssa
plt.plot(time_grid, means_fr[species_idx, :], 'r', label='Mean SSA (first reaction)')
plt.fill_between(time_grid, means_fr[species_idx,:]+stds_fr[species_idx, :], means_fr[species_idx,:]-stds_fr[species_idx, :], 
                 color='r',alpha=0.3)
plt.xlim([0, end_time])    
plt.xlabel('Time (s)')
plt.ylabel('Number of species')
plt.xlim([0, max(time_grid)])
plt.ylim([0, 1.1 * max([(means_direct[species_idx,:]+stds_direct[species_idx, :]).max(), 
                        (means_fr[species_idx,:]+stds_fr[species_idx, :]).max()])])
plt.legend()

### CPU vs GPU

In [ ]:
from circuits.auto_repressor.tetr_srna_incis import TetRsRNAInCis, cfg

cfg['solver'] = 'direct'
device = torch.device("cuda:0")
reaction_system_c = TetRsRNAInCis(device=device)
init_pops_c = torch.zeros((reaction_system_c.n_species, ), dtype=torch.int64, device=device) 
data_set_c = SimulationData(device=device)
ssa_simulator_c = StochasticSimulator(
    reaction_system=reaction_system_c,
    data_set=data_set_c,
    cfg=cfg,
    device=device
)

device = torch.device("cpu")
reaction_system = TetRsRNAInCis(device=device)
init_pops = torch.zeros((reaction_system.n_species, ), dtype=torch.int64, device=device) 
data_set = SimulationData(device=device)
ssa_simulator = StochasticSimulator(
    reaction_system=reaction_system,
    data_set=data_set,
    cfg=cfg,
    device=device
)

In [ ]:
n_traj = 100000
%timeit ssa_simulator_c.simulate(init_pops=init_pops_c, end_time=end_time, n_trajectories=n_traj)
%timeit ssa_simulator.simulate(init_pops=init_pops, end_time=end_time, n_trajectories=n_traj)